In [1]:
! pip install datasets transformers

     |████████████████████████████████| 306 kB 5.3 MB/s 
     |████████████████████████████████| 3.4 MB 58.1 MB/s 
     |████████████████████████████████| 132 kB 74.8 MB/s 
     |████████████████████████████████| 1.1 MB 64.8 MB/s 
     |████████████████████████████████| 243 kB 42.2 MB/s 
     |████████████████████████████████| 61 kB 496 kB/s 
     |████████████████████████████████| 3.3 MB 47.5 MB/s 
     |████████████████████████████████| 596 kB 53.4 MB/s 
     |████████████████████████████████| 895 kB 65.7 MB/s 
     |████████████████████████████████| 271 kB 62.9 MB/s 
     |████████████████████████████████| 192 kB 73.2 MB/s 
     |████████████████████████████████| 160 kB 51.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Przygotowanie zbioru danych i modelu

In [19]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')
#datasets = load_dataset("text", data_files={"train": 'train.txt', "validation": 'valid.txt'})

Reusing dataset wikitext (/root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
model_checkpoint = "distilgpt2"

In [21]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/distilgpt2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/f985248d2791fcff97732e4ee263617adec1edb5429a2b8421734c6d14e39bee.422318838d1ec4e061efb4ea29671cb2a044e244dc69229682bebd7cacc81631
Model config GPT2Config {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_at

In [22]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [23]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-626d7b4a61440017.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-f2b2a4d59acc8642.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-4d22d74287261357.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-94bbc74c319aebb5.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-ee32c26690ee2fdd.arrow
Loading cached 

In [24]:
# block_size = tokenizer.model_max_length
block_size = 128

In [25]:
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [26]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-066262a5dd6fc1fd.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-d2480a6a6fa11b25.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-f2b27611281740f7.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-b73fa8b904aa3395.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-d8ae239e68d995b4.arrow
Loading cached 

In [27]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

loading configuration file https://huggingface.co/distilgpt2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/f985248d2791fcff97732e4ee263617adec1edb5429a2b8421734c6d14e39bee.422318838d1ec4e061efb4ea29671cb2a044e244dc69229682bebd7cacc81631
Model config GPT2Config {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_p

In [28]:
from transformers import Trainer, TrainingArguments

In [29]:
#reimplementacja ScaleGrad
import torch.nn.functional as F
import torch
def getNovelMask(target, vocab_size):
    b, l = target.size()
    zeros = torch.zeros(b, l, vocab_size).to(target.device)
    ones = torch.ones(b, l, vocab_size).to(target.device)

    target_index = target.unsqueeze(1).expand(b, l, l).transpose(-2, -1).triu().transpose(-2, -1)
    matrix = zeros.scatter_add_(2, target_index, ones)
    matrix[:, :, 0] = 0
    summ_true = torch.tensor(range(1, l + 1)).unsqueeze(0).float().to(target.device)
    summ_now = torch.sum(matrix, dim=-1)
    diff = summ_true - summ_now
    matrix[:, :, 0] = diff
    matrix = torch.cat((torch.zeros(b, 1, vocab_size).to(target.device), matrix[:, :-1, :]), 1)
    novel_mask = matrix < 1.

    return novel_mask


def sg_loss(model, batch, gamma, output):
    longer_sample = batch['input_ids'][0]
    inp = longer_sample
    model_output = output
    target = batch['labels'][0]
    logits = model_output[1]

    # ScaleGrad
    probs = F.softmax(logits, dim=-1)
    novel_mask = getNovelMask(target.unsqueeze(0), logits.size(-1))
    rep_mask = ~novel_mask

    new_probs = probs * novel_mask * gamma + probs * rep_mask + 1e-8
    new_probs = F.normalize(new_probs, p=1, dim=-1)
    lprobs = torch.log(new_probs)
    lprobs_flatten = torch.unsqueeze(lprobs[0,:,:],0)
    assert lprobs_flatten.size(0) == 1, 'Nonflat sequence error'
    loss = F.nll_loss(lprobs_flatten[0], target, reduction='sum')
    true_token_logits = -F.nll_loss(logits[0], target, reduction='none')
    ntokens = inp.numel()

    return loss / ntokens

In [30]:
#gamma dla ScaleGrad, dla MLE: gamma =1.0
gamma = 0.2
class ScaleGradTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss = sg_loss(model, inputs, gamma, outputs)
        return (loss, outputs) if return_outputs else loss

In [31]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
    num_train_epochs = 1.0
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [32]:
trainer = ScaleGradTrainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [33]:
trainer.train()

***** Running training *****
  Num examples = 18666
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2334


Epoch,Training Loss,Validation Loss
1,0.037200,0.011782


Saving model checkpoint to distilgpt2-finetuned-wikitext2/checkpoint-500
Configuration saved in distilgpt2-finetuned-wikitext2/checkpoint-500/config.json
Model weights saved in distilgpt2-finetuned-wikitext2/checkpoint-500/pytorch_model.bin
Saving model checkpoint to distilgpt2-finetuned-wikitext2/checkpoint-1000
Configuration saved in distilgpt2-finetuned-wikitext2/checkpoint-1000/config.json
Model weights saved in distilgpt2-finetuned-wikitext2/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to distilgpt2-finetuned-wikitext2/checkpoint-1500
Configuration saved in distilgpt2-finetuned-wikitext2/checkpoint-1500/config.json
Model weights saved in distilgpt2-finetuned-wikitext2/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to distilgpt2-finetuned-wikitext2/checkpoint-2000
Configuration saved in distilgpt2-finetuned-wikitext2/checkpoint-2000/config.json
Model weights saved in distilgpt2-finetuned-wikitext2/checkpoint-2000/pytorch_model.bin
***** Running Evaluation **

TrainOutput(global_step=2334, training_loss=0.14138225458990128, metrics={'train_runtime': 32008.8344, 'train_samples_per_second': 0.583, 'train_steps_per_second': 0.073, 'total_flos': 609670643318784.0, 'train_loss': 0.14138225458990128, 'epoch': 1.0})

In [34]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1931
  Batch size = 8


Perplexity: 1.01


In [35]:
model.save_pretrained('pretrained_gpt2')

Configuration saved in pretrained_gpt2/config.json
Model weights saved in pretrained_gpt2/pytorch_model.bin
